In [1]:
import random
from time import sleep, time
from copy import copy, deepcopy

import torch
from torch.nn import *
from torch.optim import SGD, Adam

import gym

from gym.logger import ERROR
gym.logger.set_level(ERROR)

from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
import functools
from torch.utils.tensorboard import SummaryWriter
import numpy as np

from functools import partial
import ray

from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import HyperBandScheduler, PopulationBasedTraining

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import os
from typing import List
import pickle

/home/jan/.local/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [18]:
writer = SummaryWriter()

In [19]:
# writer.add_scalars("Loss", {"train": loss, "valid": val_loss}, epoch)
# writer.flush()

In [20]:
def timeit(func):
    if "performance_measurements" not in globals():
        global performance_measurements
        performance_measurements = dict()

    @functools.wraps(func)
    def newfunc(*args, **kwargs):
        start = time()
        ret_val = func(*args, **kwargs)
        end = time()
        elapsed = end - start
        
        key = f"{func.__name__}"
        if isinstance(performance_measurements.get(key), list):
            performance_measurements[key].append(elapsed)
        elif performance_measurements.get(key) is None:
            performance_measurements[key] = [elapsed]
        
        step = len(performance_measurements[key])
        writer.add_scalars("Performance", {key: elapsed}, step)
        writer.flush()
        
        return ret_val
    return newfunc

def show_times():
    if "performance_measurements" not in globals():
        global performance_measurements
        performance_measurements = dict()

    for key, value in performance_measurements.items():
        records = len(value)
        mean = sum(value)/records
        print(f"[{key}] Mean execution time: {mean}, records number: {records}")

def clear_times():
    if "performance_measurements" not in globals():
        global performance_measurements
    performance_measurements = dict()

In [21]:
class Actor(Module):
    def __init__(self, state_dim=8, action_dim=2, layer_sizes=[128, 64, 32, 16]):
        super().__init__()
        layer_sizes = [state_dim] + layer_sizes
        layers = []
        for i in range(len(layer_sizes)-1):
            layers += [
                Linear(layer_sizes[i], layer_sizes[i+1]),
                ReLU(),
                BatchNorm1d(layer_sizes[i+1])
            ]
        layers.append(Linear(layer_sizes[-1], action_dim))
        
        self.net = Sequential(*layers)
    
    def forward(self, state):
        action = self.net(state)
        return action

class Critic(Module):
    def __init__(self, state_dim=8, action_dim=2, layer_sizes=[128, 64, 32, 16]):
        super().__init__()
        layer_sizes = [state_dim+action_dim] + layer_sizes
        layers = []
        for i in range(len(layer_sizes)-1):
            layers += [
                Linear(layer_sizes[i], layer_sizes[i+1]),
                ReLU(),
                BatchNorm1d(layer_sizes[i+1])
            ]
        layers.append(Linear(layer_sizes[-1], 1))
        
        self.net = Sequential(*layers)
    
    def forward(self, state, action):
        inp = torch.cat([state, action], dim=1)
        action_value = self.net(inp)
        return action_value

In [22]:
class ReplayBuffer:
    def __init__(self, batch_size=1, max_size=None):
        self.buffer = []
        self.batch_size = batch_size
        self.max_size = max_size
    
    def __lshift__(self, record):
        self._add_record(record)
    
    def __rshift__(self, record):
        self._add_record(record)
    
    def _add_record(self, record):
        assert len(record) == 5
        record = deepcopy(record)
        state, action, reward, next_state, done = record
        
        state = torch.Tensor(state)
        action = torch.Tensor(action)
        reward = torch.Tensor([reward])
        next_state = torch.Tensor(next_state)
        done = torch.Tensor([done])
        
        record = state, action, reward, next_state, done
        
        
        if self.max_size is not None and len(self.buffer) == self.max_size:
            self.buffer.pop(0)
        self.buffer.append(record)
    
    def __iter__(self):
        return self
 
    def __next__(self):
        batch_size = min(self.batch_size, len(self.buffer))
        if batch_size == 0:
            raise Exception("Replay Buffer is Empty")
        
        raw_records = random.sample(self.buffer, k=batch_size)
        
        raw_states = []
        raw_actions = []
        raw_rewards = []
        raw_next_states = []
        raw_dones = []
        
        for record in raw_records:
            state, action, reward, next_state, done = record
            raw_states.append(state)
            raw_actions.append(action)
            raw_rewards.append(reward)
            raw_next_states.append(next_state)
            raw_dones.append(done)

        states = torch.stack(raw_states)
        actions = torch.stack(raw_actions)
        rewards = torch.stack(raw_rewards)
        next_states = torch.stack(raw_next_states)
        dones = torch.stack(raw_dones)
        
        batch = (states, actions, rewards, next_states, dones)
        
        return batch
    
    def __str__(self):
        return self.buffer.__str__()
    
    def __repr__(self):
        return self.__str__()

In [23]:
class Agent:
    def __init__(
        self,
        device="cpu",
        state_dim=8,
        action_dim=2,
        actor_layer_sizes=[64, 32, 16],
        critic_layer_sizes=[64, 32, 16],
        replay_buffer_max_size=1000,
        batch_size=64,
        learning_freq = 1,
        γ=0.99,
        μ_θ_α=0.01,
        Q_Φ_α=0.01,
        ρ=0.95,
        noise_scale=0.1,
        train_after=0,
        exploration=True,
        writer=None,
        train_steps_per_update=1,
        action_low = [-1, -1],
        action_high = [1, 1]
    ):
        self.device = device
        
        self.μ_θ = Actor(
            state_dim=state_dim,
            action_dim=action_dim,
            layer_sizes=actor_layer_sizes
        ).to(device)
        
        self.Q_Φ = Critic(
            state_dim=state_dim,
            action_dim=action_dim,
            layer_sizes=critic_layer_sizes
        ).to(device)
        
        self.μ_θ_targ = deepcopy(self.μ_θ)
        self.Q_Φ_targ = deepcopy(self.Q_Φ)
        self.Q_Φ_targ.eval()
        self.μ_θ_targ.eval()
        
        self.batch_size = batch_size
        self.Ɗ = ReplayBuffer(
            max_size=replay_buffer_max_size,
            batch_size=batch_size
        )
        self.learning_freq = learning_freq
        self.γ = γ

        self.MSE = MSELoss()

        self.μ_θ_α = μ_θ_α
        self.μ_θ_optimizer = SGD(self.μ_θ.parameters(), μ_θ_α)

        self.Q_Φ_α = Q_Φ_α
        self.Q_Φ_optimizer = SGD(self.Q_Φ.parameters(), Q_Φ_α)

        self.ρ = ρ
        self.noise_scale = noise_scale
        
        self._last_S = None
        self._last_A = None
        
        self.steps_counter = 0
        
        self.returns = []
        self._last_return = 0
        
        self.train_after = train_after
        self.exploration = exploration
        
        self.writer = writer
        
        self.train_steps_per_update = train_steps_per_update
        
        self.action_low = action_low
        self.action_high = action_high
    
    def act(self, S):
        self._last_S = S
        
        S = self._prepare_state(S)
        self.μ_θ.eval()
        with torch.no_grad():
            A = self.μ_θ(S)
            if self.exploration:
                A = self._add_noise(A)
            A = self._prepare_action(A)
        
        self._last_A = A
        
        return A
    
    def observe(self, R, S_prim, d):
        self._last_return += R
        
        if d:
            return_val = copy(self._last_return)
            self.returns.append(return_val)
            self._last_return = 0
            if self.writer is not None:
                self.writer.add_scalar("Return", return_val, self.steps_counter)
                self.writer.flush()
        
        S = self._last_S
        A = self._last_A
        
        self.Ɗ << (S, A, R, S_prim, d)
        
        self.steps_counter += 1
        
        if self._update_time():
            for _ in range(self.train_steps_per_update):
                self.train_step()
    
    def _update_time(self):
        flag = (
            self.steps_counter >= self.train_after
            and
            self.steps_counter >= self.batch_size
            and
            self.steps_counter % self.learning_freq == 0
        )
        return flag
        
    
    def train_step(self):
        batch = next(self.Ɗ)
        
        batch = tuple([t.to(self.device) for t in batch])

        self._critic_train_step(batch)
        self._actor_train_step(batch)
        self._target_nets_train_step()
    
    def _critic_train_step(self, batch):
        S, A, R, S_prim, d = batch
        
        self.μ_θ.eval()
        with torch.no_grad():
            y = R + self.γ*(1-d)*self.Q_Φ_targ(S_prim, self.μ_θ_targ(S_prim))
        self.Q_Φ.train()
        Q_Φ_ℒ = self.MSE(self.Q_Φ(S, A), y)
        Q_Φ_ℒ.backward()
        utils.clip_grad_value_(self.Q_Φ.parameters(), clip_value=0.5)
        utils.clip_grad_norm_(self.Q_Φ.parameters(), max_norm=1.0, norm_type=2.0)
        self.Q_Φ_optimizer.step()
    
    def _actor_train_step(self, batch):
        S, A, R, S_prim, d = batch
        
        self.Q_Φ.eval()
        self.μ_θ.train()
        μ_θ_ℒ = -torch.mean(self.Q_Φ(S, self.μ_θ(S))) # Minus because gradient ascent
        μ_θ_ℒ.backward()
        utils.clip_grad_value_(self.μ_θ.parameters(), clip_value=0.5)
        utils.clip_grad_norm_(self.μ_θ.parameters(), max_norm=1.0, norm_type=2.0)
        self.μ_θ_optimizer.step()
    
    def _target_nets_train_step(self):
        for Φ, Φ_targ in zip(self.Q_Φ.parameters(), self.Q_Φ_targ.parameters()):
            Φ_targ.data = self.ρ * Φ_targ.data + (1-self.ρ) * Φ.data
        for θ, θ_targ in zip(self.μ_θ.parameters(), self.μ_θ_targ.parameters()):
            θ_targ.data = self.ρ * θ_targ.data + (1-self.ρ) * θ.data
        
    def _prepare_state(self, S):
        state_batch = torch.Tensor(S).unsqueeze(0).to(self.device)
        return state_batch

    def _prepare_action(self, A):
        A = A.squeeze_(0).cpu().numpy()
        return A
    
    def _add_noise(self, action):
        action = action.cpu()
        low = torch.Tensor(self.action_low)
        high = torch.Tensor(self.action_high)
        the_range = high - low
        normal = torch.normal(0.5, 0.5*1/3, (2,))
        deviation = the_range * normal
        epsilon = deviation * self.noise_scale * (-1)**random.randint(0,1)
        noised_action = torch.max(torch.min(action+epsilon, high), low)
        
        noised_action.to(self.device)
        
        return noised_action
    
    def to(self, device):
        new_agent = deepcopy(self)
        
        new_agent.device = device
        
        if new_agent.device == "cuda":
            if torch.cuda.is_available():
                if torch.cuda.device_count() > 1:
                    new_agent.μ_θ = nn.DataParallel(new_agent.μ_θ)
                    new_agent.Q_Φ = nn.DataParallel(new_agent.Q_Φ)
            else:
                new_agent.device = "cpu"

        new_agent.μ_θ_targ = deepcopy(new_agent.μ_θ)
        new_agent.Q_Φ_targ = deepcopy(new_agent.Q_Φ)
        new_agent.μ_θ_targ.eval()
        new_agent.Q_Φ_targ.eval()

        new_agent.μ_θ.to(new_agent.device)
        new_agent.Q_Φ.to(new_agent.device)
        new_agent.μ_θ_targ.to(new_agent.device)
        new_agent.Q_Φ_targ.to(new_agent.device)

        # Optimizer has to be instantiated after moving nets to selected device
        new_agent.μ_θ_optimizer = SGD(new_agent.μ_θ.parameters(), new_agent.μ_θ_α)
        new_agent.Q_Φ_optimizer = SGD(new_agent.Q_Φ.parameters(), new_agent.Q_Φ_α)
        
        return new_agent

In [52]:
with open("somepath", "wb") as file:
    torch.save(agent, file)

In [53]:
with open("somepath", "rb") as file:
    a2 = torch.load(file)

In [55]:
a2

In [45]:
d["device"]

str

In [ ]:
self.device
self.μ_θ
self.Q_Φ
self.μ_θ_targ
self.Q_Φ_targ
self.batch_size
self.Ɗ
self.learning_freq
self.γ
self.MSE
self.μ_θ_α
self.μ_θ_optimizer
self.Q_Φ_α
self.Q_Φ_optimizer
self.ρ
self.noise_scale
self._last_S
self._last_A
self.steps_counter
self.returns
self._last_return
self.train_after
self.exploration
self.writer
self.train_steps_per_update
self.action_low
self.action_high

In [12]:
# filepath = 'somepath'
# with open(filepath, 'wb') as file:
#     pickle.dump(agent, file)

NameError: name 'agent' is not defined

In [183]:
# filepath = 'somepath'
# with open(filepath, 'rb') as file:
#     a2 = pickle.load(file)

In [164]:
# env.close()

In [2]:
env = gym.make("LunarLanderContinuous-v2")

In [6]:
env.action_space.

array([1., 1.], dtype=float32)

In [14]:
# env.action_space.low

In [35]:
action_dim = env.action_space.shape[0]
state_dim = env.observation_space.shape[0]

agent = Agent(
    device="cpu",
    state_dim=state_dim,
    action_dim=action_dim,
    actor_layer_sizes=[64, 32, 16],
    critic_layer_sizes=[64, 32, 16],
    replay_buffer_max_size=10000,
    batch_size=64,
    learning_freq = 64,
    γ=0.99,
    μ_θ_α=10e-6,
    Q_Φ_α=10e-6,
    ρ=0.95,
    noise_scale=0.1,
    train_after=64,
    exploration=True,
    writer = None,
    train_steps_per_update=32
)

In [33]:
with open("somepath", "wb") as file:
    pickle.dump(agent, file)

In [34]:
with open("somepath", "rb") as file:
    agent = pickle.load(file)

In [16]:
# t = torch.Tensor([1,2,3])
# print(t)
# a = torch.Tensor(t)
# print(a)

In [17]:
# Segmentation fault !!!
# t = torch.Tensor([1,2,3]).to("cuda")
# print(t)
# a = torch.Tensor(t)
# print(a)

In [18]:
# agent.μ_θ_optimizer = SGD(agent.μ_θ.parameters(), 10e-3)
# agent.Q_Φ_optimizer = SGD(agent.Q_Φ.parameters(), 10e-3)

In [26]:
def simulate(env, agent, steps, render=True):
    S = env.reset()
    for i in range(steps):
        if render:
            timeit(env.render)()
            
        A = timeit(agent.act)(S)

        S_prim, R, d, _ = timeit(env.step)(A)

        timeit(agent.observe)(R, S_prim, d)

        S = S_prim

        if d:
            S = env.reset()

    env.close()

In [30]:
env.close()

In [29]:
simulate(env, agent.to("cpu").to("cuda"), steps=300, render=True)

In [17]:
# agent.noise_scale = 0.1

## Hyperparameters tuning

In [18]:
# # Set device
# device = "cpu"
# if torch.cuda.is_available():
#     device = "cuda:0"
#     if torch.cuda.device_count() > 1:
#         agent.μ_θ = nn.DataParallel(agent.μ_θ)
#         agent.Q_Φ = nn.DataParallel(agent.Q_Φ)

# agent.device = device

# agent.μ_θ_targ = deepcopy(agent.μ_θ)
# agent.Q_Φ_targ = deepcopy(agent.Q_Φ)
# agent.μ_θ_targ.eval()
# agent.Q_Φ_targ.eval()

# agent.μ_θ.to(device)
# agent.Q_Φ.to(device)
# agent.μ_θ_targ.to(device)
# agent.Q_Φ_targ.to(device)

# # Optimizer has to be instantiated after moving nets to selected device
# agent.μ_θ_optimizer = SGD(agent.μ_θ.parameters(), agent.μ_θ_α)
# agent.Q_Φ_optimizer = SGD(agent.Q_Φ.parameters(), agent.Q_Φ_α)

# agent.Ɗ.device=device

In [19]:
# show_times()

[act] Mean execution time: 0.0006904387474060058, records number: 300
[step] Mean execution time: 0.0002632594108581543, records number: 300
[observe] Mean execution time: 0.0033260027567545572, records number: 300


In [20]:
# def layer_size_sampler() -> List[int]:
#     layers_no = random.randint(3,5)
#     layer_sizes = []
#     for _ in range(layers_no):
#         layer_sizes.append(2**random.randint(2, 9))
#     return layer_sizes

# layer_size_sampler()

[512, 8, 128]

In [56]:
episodes_n = 1000

config = {
    "actor_layer_0_size": tune.choice([2**x for x in range(0,10)]),
    "actor_layer_1_size": tune.choice([2**x for x in range(0,10)]),
    "actor_layer_2_size": tune.choice([2**x for x in range(0,10)]),
    "actor_layer_3_size": tune.choice([2**x for x in range(0,10)]),
    "actor_layer_4_size": tune.choice([2**x for x in range(0,10)]),
    
    "critic_layer_0_size": tune.choice([2**x for x in range(0,10)]),
    "critic_layer_1_size": tune.choice([2**x for x in range(0,10)]),
    "critic_layer_2_size": tune.choice([2**x for x in range(0,10)]),
    "critic_layer_3_size": tune.choice([2**x for x in range(0,10)]),
    "critic_layer_4_size": tune.choice([2**x for x in range(0,10)]),
    
    "replay_buffer_max_size": tune.choice([10**x for x in range(2,8)]),
    "batch_size": tune.choice([2**x for x in range(2,12)]),
    "learning_freq": tune.choice([2**x for x in range(2,9)]),
    "γ": tune.uniform(1e-2, 1),
    "μ_θ_α": tune.loguniform(1e-6, 1e-1),
    "Q_Φ_α": tune.loguniform(1e-6, 1e-1),
    "ρ": tune.loguniform(0.5, 0.95),
    "noise_scale": tune.uniform(0, 1),
    "train_steps_per_update": tune.choice([2**x for x in range(0,6)]),
}

class AgentTrainable(tune.Trainable):
    def setup(self, config):
        # Instantiate environment and agent
        self.env = gym.make("LunarLanderContinuous-v2")

        action_dim = self.env.action_space.shape[0]
        state_dim = self.env.observation_space.shape[0]
        
        actor_layer_sizes = list(filter(
            lambda x: x > 0,
            [config[f"actor_layer_{i}_size"] for i in range(0,5)]
        ))
        
        if actor_layer_sizes == []:
            actor_layer_sizes = [2]
        
        
        critic_layer_sizes = list(filter(
            lambda x: x > 0,
            [config[f"critic_layer_{i}_size"] for i in range(0,5)]
        ))
        
        if critic_layer_sizes == []:
            critic_layer_sizes = [2]
        
        self.agent = Agent(
            device="cuda",
            state_dim=state_dim,
            action_dim=action_dim,
            actor_layer_sizes=actor_layer_sizes,
            critic_layer_sizes=critic_layer_sizes,
            replay_buffer_max_size=config["replay_buffer_max_size"],
            batch_size=config["batch_size"],
            learning_freq=config["learning_freq"],
            γ=config["γ"],
            μ_θ_α=config["μ_θ_α"],
            Q_Φ_α=config["Q_Φ_α"],
            ρ=config["ρ"],
            noise_scale=config["noise_scale"],
            train_after=1024,
            exploration=True,
            train_steps_per_update=config["train_steps_per_update"],
        )

# #       Set device
#         device = "cpu"
#         if torch.cuda.is_available():
#             device = "cuda"
#             if torch.cuda.device_count() > 1:
#                 self.agent.μ_θ = nn.DataParallel(self.agent.μ_θ)
#                 self.agent.Q_Φ = nn.DataParallel(self.agent.Q_Φ)

#         self.agent.μ_θ_targ = deepcopy(self.agent.μ_θ)
#         self.agent.Q_Φ_targ = deepcopy(self.agent.Q_Φ)
#         self.agent.μ_θ_targ.eval()
#         self.agent.Q_Φ_targ.eval()

#         self.agent.μ_θ.to(device)
#         self.agent.Q_Φ.to(device)
#         self.agent.μ_θ_targ.to(device)
#         self.agent.Q_Φ_targ.to(device)

#         # Optimizer has to be instantiated after moving nets to selected device
#         self.agent.μ_θ_optimizer = SGD(self.agent.μ_θ.parameters(), config["μ_θ_α"])
#         self.agent.Q_Φ_optimizer = SGD(self.agent.Q_Φ.parameters(), config["Q_Φ_α"])
        self.agent = agent.to("cuda")

    def step(self):
        d = False
        S = self.env.reset()
        for episod_i in range(episodes_n):
            while not d:
                A = self.agent.act(S)
                S_prim, R, d, _ = self.env.step(A)
                self.agent.observe(R, S_prim, d)
                S = S_prim
                
        last_x_episodes = int(0.1*episodes_n)
        mean_return = torch.mean(torch.Tensor(self.agent.returns[-last_x_episodes:])).item()
        
        return {"mean_return": mean_return}
    
    def cleanup(self):
        self.env.close()
    
    def save_checkpoint(self, tmp_checkpoint_dir):
        path = os.path.join(tmp_checkpoint_dir, "checkpoint")
        # First idea
#         torch.save(
#             (
#                 self.agent.μ_θ.state_dict(),
#                 self.agent.μ_θ_optimizer.state_dict(),
#                 self.agent.Q_Φ.state_dict(),
#                 self.agent.Q_Φ_optimizer.state_dict(),
#             ),
#             path
#         )
        # Second idea
#         with open(path, 'wb') as file:
#             pickle.dump(self.agent, file)
        # Third idea
        torch.save(self.agent, path)   
            
        return tmp_checkpoint_dir

    def load_checkpoint(self, tmp_checkpoint_dir):
        path = os.path.join(tmp_checkpoint_dir, "checkpoint")
# First idea
#         μ_θ_state, μ_θ_optimizer_state, Q_Φ_state, Q_Φ_optimizer_state = torch.load(
#             path
#         )
#         self.agent.μ_θ.load_state_dict(μ_θ_state)
#         self.agent.μ_θ_optimizer.load_state_dict(μ_θ_optimizer_state)
#         self.agent.Q_Φ.load_state_dict(Q_Φ_state)
#         self.agent.Q_Φ_optimizer.load_state_dict(Q_Φ_optimizer_state)
# Second idea
#         with open(path, 'rb') as file:
#             self.agent = pickle.load(file)
# Third idea
        self.agent = torch.load(path)


In [82]:
# hyperopt = HyperOptSearch(metric="mean_return", mode="max")

In [45]:
# hyperband = HyperBandScheduler(metric="mean_return", mode="max")

In [58]:
CPU_N = 12
GPU_N = 1

PERC_SYSTEM_LOAD = 0.8

MAX_CPU_N = PERC_SYSTEM_LOAD*CPU_N
MAX_GPU_N = PERC_SYSTEM_LOAD*GPU_N

TRIALS_N = 4

# cpu_per_trial = MAX_CPU_N/TRIALS_N
cpu_per_trial = 1
gpu_per_trial = MAX_GPU_N/TRIALS_N

print(cpu_per_trial)
print(gpu_per_trial)

1
0.2


In [59]:
pbt = PopulationBasedTraining(
    time_attr="training_iteration",
    perturbation_interval=4,
    hyperparam_mutations=config
)

analysis = tune.run(
    AgentTrainable,
    config=config,
    scheduler=pbt,
    metric="mean_return",
    mode="max",
    fail_fast=True,
    stop={"training_iteration": 3000, "mean_return": 100},
    num_samples=TRIALS_N,
    resources_per_trial={'cpu':cpu_per_trial, 'gpu': gpu_per_trial},
    local_dir="./results"
)

Trial name,status,loc,Q_Φ_α,actor_layer_0_size,actor_layer_1_size,actor_layer_2_size,actor_layer_3_size,actor_layer_4_size,batch_size,critic_layer_0_size,critic_layer_1_size,critic_layer_2_size,critic_layer_3_size,critic_layer_4_size,learning_freq,noise_scale,replay_buffer_max_size,train_steps_per_update,γ,μ_θ_α,ρ
Trainable_467c2_00000,PENDING,,5.04955e-05,512,1,4,1,64,64,1,16,4,512,2,128,0.936652,10000000,32,0.33683,1.53819e-05,0.7955
Trainable_467c2_00001,PENDING,,0.0173705,16,32,2,4,16,8,4,4,128,256,32,32,0.983955,10000000,32,0.598748,0.000563532,0.674373
Trainable_467c2_00002,PENDING,,0.0395119,8,16,2,8,8,8,2,2,512,4,128,4,0.514658,10000,8,0.295667,0.0741235,0.814801
Trainable_467c2_00003,PENDING,,0.000551092,128,64,8,512,512,16,16,128,4,64,2,16,0.297513,10000,16,0.164244,0.0136527,0.663534


(raylet) /home/jan/.local/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /home/jan/.local/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /home/jan/.local/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(r

Result for Trainable_467c2_00000:
  date: 2021-06-17_13-58-09
  done: false
  experiment_id: d65940735a8d4d4b84ea8228a4fcbae2
  hostname: DESKTOP-Jan
  iterations_since_restore: 1
  mean_return: -104.43452453613281
  node_ip: 192.168.0.19
  pid: 11537
  time_since_restore: 0.0848841667175293
  time_this_iter_s: 0.0848841667175293
  time_total_s: 0.0848841667175293
  timestamp: 1623931089
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 467c2_00000
  
Result for Trainable_467c2_00002:
  date: 2021-06-17_13-58-09
  done: false
  experiment_id: 06ff6c8ab523434fb1087623c325fadf
  hostname: DESKTOP-Jan
  iterations_since_restore: 1
  mean_return: -119.66719818115234
  node_ip: 192.168.0.19
  pid: 11536
  time_since_restore: 0.07181978225708008
  time_this_iter_s: 0.07181978225708008
  time_total_s: 0.07181978225708008
  timestamp: 1623931089
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 467c2_00002
  
Result for Trainable_467c2_00003:
  date: 2021-06-17_13

Trial name,status,loc,Q_Φ_α,actor_layer_0_size,actor_layer_1_size,actor_layer_2_size,actor_layer_3_size,actor_layer_4_size,batch_size,critic_layer_0_size,critic_layer_1_size,critic_layer_2_size,critic_layer_3_size,critic_layer_4_size,learning_freq,noise_scale,replay_buffer_max_size,train_steps_per_update,γ,μ_θ_α,ρ,iter,total time (s),mean_return
Trainable_467c2_00000,RUNNING,192.168.0.19:11537,5.04955e-05,512,1,4,1,64,64,1,16,4,512,2,128,0.936652,10000000,32,0.33683,1.53819e-05,0.7955,2,1.84107,-209.939
Trainable_467c2_00001,RUNNING,192.168.0.19:11533,0.0173705,16,32,2,4,16,8,4,4,128,256,32,32,0.983955,10000000,32,0.598748,0.000563532,0.674373,2,1.29785,-234.244
Trainable_467c2_00002,RUNNING,192.168.0.19:11536,0.0395119,8,16,2,8,8,8,2,2,512,4,128,4,0.514658,10000,8,0.295667,0.0741235,0.814801,2,1.24241,-189.85
Trainable_467c2_00003,RUNNING,192.168.0.19:11535,0.000551092,128,64,8,512,512,16,16,128,4,64,2,16,0.297513,10000,16,0.164244,0.0136527,0.663534,2,1.25267,-196.254


Result for Trainable_467c2_00000:
  date: 2021-06-17_13-58-14
  done: false
  experiment_id: d65940735a8d4d4b84ea8228a4fcbae2
  hostname: DESKTOP-Jan
  iterations_since_restore: 7
  mean_return: -251.64990234375
  node_ip: 192.168.0.19
  pid: 11537
  time_since_restore: 5.356344699859619
  time_this_iter_s: 0.5713541507720947
  time_total_s: 5.356344699859619
  timestamp: 1623931094
  timesteps_since_restore: 0
  training_iteration: 7
  trial_id: 467c2_00000
  
Result for Trainable_467c2_00003:
  date: 2021-06-17_13-58-14
  done: false
  experiment_id: 4b5aa05109484827a8dd9f65019c7be0
  hostname: DESKTOP-Jan
  iterations_since_restore: 6
  mean_return: -196.2876739501953
  node_ip: 192.168.0.19
  pid: 11535
  time_since_restore: 5.349221706390381
  time_this_iter_s: 0.6046538352966309
  time_total_s: 5.349221706390381
  timestamp: 1623931094
  timesteps_since_restore: 0
  training_iteration: 6
  trial_id: 467c2_00003
  
Result for Trainable_467c2_00001:
  date: 2021-06-17_13-58-14
  do

...

2021-06-17 14:04:59,763	WARNING tune.py:507 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Result for Trainable_467c2_00003:
  date: 2021-06-17_14-04-59
  done: false
  experiment_id: 5b7c50ce549b48bdae4ffa7a816fc9bd
  hostname: DESKTOP-Jan
  iterations_since_restore: 136
  mean_return: -237.00277709960938
  node_ip: 192.168.0.19
  pid: 12488
  time_since_restore: 116.99356698989868
  time_this_iter_s: 1.1368153095245361
  time_total_s: 116.99356698989868
  timestamp: 1623931499
  timesteps_since_restore: 0
  training_iteration: 136
  trial_id: 467c2_00003
  


Trial name,status,loc,Q_Φ_α,actor_layer_0_size,actor_layer_1_size,actor_layer_2_size,actor_layer_3_size,actor_layer_4_size,batch_size,critic_layer_0_size,critic_layer_1_size,critic_layer_2_size,critic_layer_3_size,critic_layer_4_size,learning_freq,noise_scale,replay_buffer_max_size,train_steps_per_update,γ,μ_θ_α,ρ,iter,total time (s),mean_return
Trainable_467c2_00000,RUNNING,192.168.0.19:12628,1.95798e-06,1,72,0,4,1,6,0,1,244,529,7,28,0.906813,6144000,38,0.827709,0.0229366,0.621502,69,60.005,-302.347
Trainable_467c2_00001,RUNNING,192.168.0.19:11534,2.44748e-06,64,91,0,0,128,5,0,1,204,441,9,24,1.13352,5120000,32,0.689758,0.0191139,0.776878,369,329.068,-317.733
Trainable_467c2_00002,RUNNING,192.168.0.19:12747,2.93697e-06,76,72,2,0,102,4,0,0,244,529,7,28,0.906813,4096000,38,0.827709,0.0152911,0.621502,45,32.8788,-345.33
Trainable_467c2_00003,RUNNING,192.168.0.19:12488,2.93697e-06,51,109,0,0,102,8,16,1,163,256,7,19,0.523356,4096000,38,0.443716,0.015433,0.642412,136,116.994,-237.003


(pid=11534) 2021-06-17 14:04:59,783	ERROR worker.py:382 -- SystemExit was raised from the worker
(pid=11534) Traceback (most recent call last):
(pid=11534)   File "python/ray/_raylet.pyx", line 599, in ray._raylet.task_execution_handler
(pid=11534)   File "python/ray/_raylet.pyx", line 451, in ray._raylet.execute_task
(pid=11534)   File "python/ray/_raylet.pyx", line 488, in ray._raylet.execute_task
(pid=11534)   File "python/ray/_raylet.pyx", line 495, in ray._raylet.execute_task
(pid=11534)   File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
(pid=11534)   File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
(pid=11534)   File "/home/jan/.local/lib/python3.7/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
(pid=11534)     return method(__ray_actor, *args, **kwargs)
(pid=11534)   File "/home/jan/.local/lib/python3.7/site-packages/ray/tune/trainable.py", line 173, in train_buffered
(pid=11534)     re

2021-06-17 14:04:59,986	ERROR tune.py:545 -- Trials did not complete: [Trainable_467c2_00000, Trainable_467c2_00001, Trainable_467c2_00002, Trainable_467c2_00003]
2021-06-17 14:04:59,987	INFO tune.py:549 -- Total run time: 414.40 seconds (414.15 seconds for the tuning loop).
2021-06-17 14:04:59,987	WARNING tune.py:554 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


In [145]:
print("Best config: ", analysis.get_best_config(metric="mean_return", mode="max"))

# Get a dataframe for analyzing trial results.
df = analysis.dataframe()

2021-06-16 22:03:32,577	WARNING experiment_analysis.py:210 -- Couldn't read config from 7 paths


Best config:  {'actor_layer_0_size': 1, 'actor_layer_1_size': 19, 'actor_layer_2_size': 512, 'actor_layer_3_size': 368, 'actor_layer_4_size': 245, 'critic_layer_0_size': 350, 'critic_layer_1_size': 128, 'critic_layer_2_size': 0, 'critic_layer_3_size': 30, 'critic_layer_4_size': 1, 'replay_buffer_max_size': 768000, 'batch_size': 32, 'learning_freq': 32, 'γ': 38, 'μ_θ_α': 0.00012257927210719585, 'Q_Φ_α': 0.09710824017462356, 'ρ': 0.7148519181127572, 'noise_scale': 0.13649334533753285, 'train_steps_per_update': 0}


In [147]:
df.iloc[0]

mean_return                                                                18.6986
done                                                                         False
timesteps_total                                                                NaN
episodes_total                                                                 NaN
training_iteration                                                            2405
experiment_id                                     e07e08de2aa748c5b870244cd7b833c7
date                                                           2021-06-16_19-15-36
timestamp                                                               1623863736
time_this_iter_s                                                          0.133427
time_total_s                                                               480.824
pid                                                                          43466
hostname                                                               DESKTOP-Jan
node

In [ ]:
gpus_per_trial = 1
max_num_epochs = 10

# scheduler = ASHAScheduler(
#     time_attr="training_iteration",
#     metric="mean_return",
#     mode="max",
#     max_t=max_num_epochs,
#     grace_period=1,
#     reduction_factor=2
# )
reporter = CLIReporter(
    # parameter_columns=["l1", "l2", "lr", "batch_size"],
    metric_columns=["mean_return", "training_iteration"])
    
result = tune.run(
    partial(train, data_dir=None),
    resources_per_trial={"cpu": 1, "gpu": 1},
    config=config,
    num_samples=10,
    scheduler=scheduler,
    progress_reporter=reporter)